In [1]:
# import the modules
import pandas as pd
import datetime as dt
import numpy as np
import os
import math
import influxdb
import scipy.stats as st
import plotly.graph_objects as go

In [2]:
# define the database connection string
DB_HOST = 'localhost' 
DB_PORT = 8086
DB_NAME = 'meteorology'
DB_USER = 'root'
DB_PASSWORD = 'root'

client = influxdb.DataFrameClient(DB_HOST, DB_PORT, DB_USER, DB_PASSWORD, DB_NAME)
station_name = "mythenquai"

In [3]:
def get_data_in_current_time_period(data, days_back, days_forward):
    
    last_entry = data.iloc[-1:]
    month_back = int((last_entry.index[0] + dt.timedelta(days=-days_back)).strftime('%-m'))
    month_forward = int((last_entry.index[0] + dt.timedelta(days=days_forward)).strftime('%-m'))
    day_back = int((last_entry.index[0] + dt.timedelta(days=-days_back)).strftime('%-d'))
    day_forward = int((last_entry.index[0] + dt.timedelta(days=days_forward)).strftime('%-d'))
    #print('month_back: {}'.format(month_back))
    #print('day_back: {}'.format(day_back))
    #print('month_forward: {}'.format(month_forward))
    #print('day_forward: {}'.format(day_forward))

    df_filtered = data[
        ((data.index.month >= month_back) & (data.index.day >= day_back)
        |
        (data.index.month <= month_forward) & (data.index.day <= day_forward))
    ]
    
    #print(df_filtered)

    return df_filtered


def remove_outliers(df_in, col_name):
    
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3 - q1 # Interquartile range
    fence_low  = q1 - 1.5 * iqr
    fence_high = q3 + 1.5 * iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    
    return df_out


def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier


def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier